In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('.')))
from src.data_connection import DatabaseConnector

db_connector = DatabaseConnector()

In [ ]:
# Vancouverの現地時間で期間を設定
start_date_local = '20251217'
end_date_local = '20251224'

base_df = db_connector.read_sql(f"""
select 
base.*,
COALESCE(alerts.has_active_alert, 0) as has_active_alert,
COALESCE(alerts.high_impact_alert_count, 0) as high_impact_alert_count,
COALESCE(alerts.alert_police_activity, 0) as alert_police_activity,
COALESCE(alerts.alert_construction, 0) as alert_construction,
COALESCE(alerts.alert_technical_problem, 0) as alert_technical_problem,
COALESCE(alerts.alert_effect_no_service, 0) as alert_effect_no_service,
COALESCE(alerts.alert_effect_detour, 0) as alert_effect_detour,
COALESCE(alerts.alert_severity_score, 0) as alert_severity_score
from 
gtfs_realtime.gtfs_rt_base_forML_v base
LEFT JOIN gtfs_realtime.gtfs_rt_alert_features_route_hour_v alerts
    ON alerts.route_id = base.route_id
    AND alerts.alert_hour = base.time_bucket
where base.start_date >= '{start_date_local}' and base.start_date <= '{end_date_local}';
""")
base_df.to_csv(f'data/base_data/analyze_delay_base_{start_date_local}_{end_date_local}.csv', index=False)

In [ ]:
print('Data loaded. Number of rows:', len(base_df))
print('Date range:', base_df['start_date'].min(), 'to', base_df['start_date'].max())

In [ ]:
base_df.groupby('route_id').size().sort_values(ascending=False).head(10)